In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from scripts import process
from scripts import analyse

In [2]:
file_name = 'survey'
data_dir = 'data/SURVEY'
output_dir = 'output'

df, df_raw, dict_path = process.clean(file_name, data_dir, output_dir)

File name                : survey
Path to data dictionary  : data/SURVEY/data_dict-survey.xlsx
Path to cleaned dataset  : output/survey_cleaned.csv


Validity Test

In [3]:
test_df = df[df['prov'] == 'รพ.นพรัตน์']
questions = [
    '1669_dur', '1669_suggestion', '1669_rating',
    'amb_notify', 'amb_suggestion', 'amb_clean', 'amb_engagement', 
    'amb_rating_safety', 'amb_rating_tools', 'amb_rating_firstaid', 'amb_rating_inform', 'amb_rating_polite', 'amb_rating_driving', 'amb_rating_overall', 
    'er_rating', 'er_trust', 'er_notify', 'er_help', 'er_inform', 'er_cause', 'er_result', 'er_recover',
    'walkin_reason', 'ems_trust'
    ]

filtered_questionnaire = test_df[questions]
filtered_questionnaire_encoded = filtered_questionnaire.apply(lambda x: pd.factorize(x)[0])

cronbach_alpha_value = analyse.cronbach_alpha(filtered_questionnaire_encoded)

Cronbach's alpha = {cronbach_alpha_value}

Analysis

In [4]:
columns = pd.read_excel(dict_path, sheet_name="validate")

for _, row in columns.iterrows():
    col = row['Column Name']
    data_type = row['Data Type']
    map = row['Map']
    min = row['Min']
    
    if data_type == 'object' and map == True:
        df = process.mapping(df, col)
    elif data_type in ['float64', 'timedelta64[ns]']:
        if pd.notna(min):
            df[col] = df[col].apply(lambda x: x if x >= min else None)
        df = process.outlier_percentile(df, col, percentile=99, direction='more than')

from datetime import datetime    
shift_time = lambda x: ('night' if x < 8 else 'morning' if x < 16 else 'afternoon')
df['shift'] = df['time_hos'].apply(lambda x: shift_time(datetime.strptime(x, '%I:%M:%S %p').hour) if pd.notna(x) else None)

df['health_score'] = df.apply(lambda row: 
    (1 - (row['health_walk'] + row['health_self'] + row['health_act'] + row['health_pain'] + row['health_depress'])), 
    axis=1)

df_survey = df.copy()

In [5]:
df_analyse = df_survey.copy()

df_analyse = df_analyse[(df_analyse['prov'] != 'รพ.นพรัตน์') & (df_analyse['triage_er'].isin(['แดง', 'เหลือง']))]

prov_transfer = ['รพ.แพร่', 'รพ.มหาสารคาม', 'รพ.เจ้าพระยายมราช', 'รพ.สงขลา']

df_analyse['transfer'] = df_analyse['prov'].apply(lambda x:'yes' if x in prov_transfer else 'no')
df_analyse['age_gr'] = df_analyse['age'].apply(lambda x: '> 60' if x > 60 else '<= 60')

In [6]:
group_by = 'transfer'

table_name = 'Survey: Table 1'
parameters = ['group', 'age', 'gender', 'nationality', 'inscl', 'shift', 'trauma']
analyse.describe(df_analyse, parameters, group_by, overall=True, bootstrap=False, table_name=table_name)

Survey: Table 1
Testing for age using Mann-Whitney U
Characteristics    Total          yes            no             p-value
-----------------  -------------  -------------  -------------  ---------
group                                                           0.00
WALKIN             209            110            99
                   (47.50%)       (56.12%)       (40.57%)
EMS                231            86             145
                   (52.50%)       (43.88%)       (59.43%)
age                54.00          53.00          55.00          0.49
                   (30.75-70.00)  (28.75-70.00)  (32.00-70.25)
gender                                                          0.00
หญิง                210            109            101
                   (47.73%)       (55.61%)       (41.39%)
ชาย                230            87             143
                   (52.27%)       (44.39%)       (58.61%)
nationality                                                     0.20
Thai              

In [7]:
group_by = 'transfer'

table_name = 'Survey: Table 2'
parameters = ['health_er', 'health_ems', 'health_chronic', 'health_score']
analyse.describe(df_analyse, parameters, group_by, overall=True, bootstrap=False, table_name=table_name)

Survey: Table 2
Testing for health_score using Mann-Whitney U
Characteristics       Total        yes          no           p-value
--------------------  -----------  -----------  -----------  ---------
health_er                                                    0.44
1-2 ครั้ง               118          57           61
                      (27.57%)     (29.69%)     (25.85%)
ไม่เคย                 195          81           114
                      (45.56%)     (42.19%)     (48.31%)
มากกว่าหรือเท่ากับ 3 ครั้ง  115          54           61
                      (26.87%)     (28.12%)     (25.85%)
health_ems                                                   0.29
ไม่เคย                 309          132          177
                      (71.03%)     (67.35%)     (74.06%)
1-2 ครั้ง               65           34           31
                      (14.94%)     (17.35%)     (12.97%)
มากกว่าหรือเท่ากับ 3 ครั้ง  61           30           31
                      (14.02%)     (15.31%)     (12.97%

In [8]:
group_by = 'transfer'

table_name = 'Survey: Table 3'
parameters = ['1669_dur', '1669_suggestion', '1669_rating']
analyse.describe(df_analyse, parameters, group_by, overall=True, bootstrap=False, table_name=table_name)

Survey: Table 3
Testing for 1669_rating using Mann-Whitney U
Characteristics    Total        yes          no           p-value
-----------------  -----------  -----------  -----------  ---------
1669_dur                                                  0.00
ต่อสายได้ทันที         117          25           92
                   (75.97%)     (42.37%)     (96.84%)
รอโอนสาย           37           34           3
                   (24.03%)     (57.63%)     (3.16%)
1669_suggestion                                           0.33
ได้รับคำแนะนำ        136          50           86
                   (90.07%)     (86.21%)     (92.47%)
ไม่ได้รับคำแนะนำ      15           8            7
                   (9.93%)      (13.79%)     (7.53%)
1669_rating        5.00         5.00         5.00         0.07
                   (5.00-6.00)  (5.00-6.00)  (5.00-6.00)


In [9]:
group_by = 'transfer'

table_name = 'Survey: Table 4'
parameters = ['amb_notify', 'amb_suggestion', 'amb_clean', 'amb_engagement', 'amb_rating_safety', 'amb_rating_tools', 'amb_rating_firstaid', 'amb_rating_inform', 'amb_rating_polite', 'amb_rating_driving', 'amb_rating_overall']
analyse.describe(df_analyse, parameters, group_by, overall=True, bootstrap=False, table_name=table_name)

Survey: Table 4
Testing for amb_clean using Mann-Whitney U
Testing for amb_rating_safety using Mann-Whitney U
Testing for amb_rating_tools using Mann-Whitney U
Testing for amb_rating_firstaid using Mann-Whitney U
Testing for amb_rating_inform using Mann-Whitney U
Testing for amb_rating_polite using Mann-Whitney U
Testing for amb_rating_driving using Mann-Whitney U
Testing for amb_rating_overall using Mann-Whitney U
Characteristics      Total        yes          no           p-value
-------------------  -----------  -----------  -----------  ---------
amb_notify                                                  0.20
มาตามเวลาที่แจ้ง        164          61           103
                     (79.61%)     (74.39%)     (83.06%)
ไม่ได้รับการแจ้ง         17           10           7
                     (8.25%)      (12.20%)     (5.65%)
มาช้ากว่าเวลาที่แจ้ง      25           11           14
                     (12.14%)     (13.41%)     (11.29%)
amb_suggestion                                   

In [11]:
group_by = 'transfer'

table_name = 'Survey: Table 5'
parameters = ['er_rating', 'er_trust', 'er_notify', 'er_help', 'er_inform', 'er_cause', 'er_result', 'er_recover']
analyse.describe(df_analyse, parameters, group_by, overall=True, bootstrap=False, table_name=table_name)

Survey: Table 5
Testing for er_rating using Mann-Whitney U
Characteristics    Total        yes          no           p-value
-----------------  -----------  -----------  -----------  ---------
er_rating          5.00         5.00         5.00         0.00
                   (5.00-6.00)  (5.00-6.00)  (5.00-6.00)
er_trust                                                  0.05
เลือกใช้บริการ        415          191          224
                   (98.81%)     (97.45%)     (100.00%)
ไม่เลือกใช้บริการ      5            5            0
                   (1.19%)      (2.55%)      (0.00%)
er_notify                                                 0.82
มี                  331          153          178
                   (78.81%)     (78.06%)     (79.46%)
ไม่มี                89           43           46
                   (21.19%)     (21.94%)     (20.54%)
er_help                                                   0.02
ไม่มี                233          121          112
                   (55.48%) 

In [7]:
group_by = 'transfer'

table_name = 'Survey: Table 6'
parameters = ['ems_trust', 'walkin_reason']
analyse.describe(df_analyse, parameters, group_by, overall=True, bootstrap=False, table_name=table_name)

Survey: Table 6
Characteristics                     Total     yes       no        p-value
----------------------------------  --------  --------  --------  ---------
ems_trust                                                         0.01
เลือกใช้บริการ                         344       142       202
                                    (78.18%)  (72.45%)  (82.79%)
ไม่เลือกใช้บริการ                       96        54        42
                                    (21.82%)  (27.55%)  (17.21%)
walkin_reason                                                     0.43
ท่านคิดว่าการเดินทางมาด้วยตนเองสะดวกกว่า  122       65        57
                                    (54.95%)  (58.56%)  (51.35%)
ท่านคิดว่าเป็นการเจ็บป่วยเล็กน้อย           33        19        14
                                    (14.86%)  (17.12%)  (12.61%)
เป็นช่วงเวลาคับขันจึงไม่ได้นึกถึงรถพยาบาล    26        11        15
                                    (11.71%)  (9.91%)   (13.51%)
ท่านคิดว่าการใช้บริการรถฉุกเฉินมีความล่าช